### Introduction

In [50]:
import pandas as pd

import spanish_elections as sp # should already be installed

### Reading November 2019 Election Data

Notice that the codebook for all the data is in generate_data/README.md. In this section, I will simply use the shortcut functions within the `data` module to read the data.

#### Extract general data about provinces

First, I wish to extract some general data about each province, such as its name and the number of seats allocated in the November 2019 elections. We can do that by invoking the `load_general_data` function:

In [51]:
general_data = \
sp.data.load_general_data()

general_data.head()

,comunidad,código de provincia,provincia,población,número de mesas,censo electoral sin cera,censo cera,total censo electoral,solicitudes voto cera aceptadas,total votantes cer,total votantes cera,total votantes,votos válidos,votos a candidaturas,votos en blanco,votos nulos,diputados
0,Andalucía,4,Almería,709340,809,460639,41988,502627,2923,303481,1933,305414,302424,299763,2661,2990,6
1,Andalucía,11,Cádiz,1238714,1520,973238,29057,1002295,3485,621965,2230,624195,616079,606858,9221,8116,9
2,Andalucía,14,Córdoba,785240,935,630033,18308,648341,2358,450124,1651,451775,444376,438971,5405,7399,6
3,Andalucía,18,Granada,912075,1100,704847,50160,755007,4827,487734,3168,490902,482779,478251,4528,8123,7
4,Andalucía,21,Huelva,519932,650,391497,7522,399019,914,254766,563,255329,250681,247336,3345,4648,5


Where the key columns are `provincia` (province) and `diputados` (seats per province). In the future, I may translate the column names and add some documentation on what each variable means. 

Anyhow, let us now check the dimensions of this dataframe:

In [52]:
general_data.shape

(52, 17)

It has 52 rows: one per province in Spain.

#### Extract results of November 2019 elections

With this purpose, we will use a different function, `load_results` that accepts two strings as argument, either 'votes' or '

In [53]:
results = \
sp.data.load_results('votes')

results.head()

,provincia,party,votes
0,Almería,PSOE,89295
1,Cádiz,PSOE,188271
2,Córdoba,PSOE,146761
3,Granada,PSOE,160190
4,Huelva,PSOE,91656


Here we see that each row provides the following information:
- provincia: province
- party: political party
- votes: # of votes obtained by each political party in each province

In [54]:
results.shape

(3484, 3)

We can already find out how many political parties presented themselves to the general election in November 2019:

In [55]:
results.shape[0] // 52  # 52 is the number of provinces in Spain

67

There were 67 political parties taking part in the event.

In the notebook `explore_the_data`, I will walk you through some interesting random facts that we can already gather from this data, such as the most-populated provinces in Spain or where did a party gain the highest percentage of casted votes.

### Simulate the Results of the Spanish Elections

In this section, we want to obtain the number of seats obtained by each political party in each province from voting data (of course, it would be trivial to find it directly from the dataframe `results_by_province` as it already has this information!)

To simulate the results, we only need to call `dhondt_rule_long`. The name of this function is no coincidence: [d'Hondt rule](https://en.wikipedia.org/wiki/D%27Hondt_method) is the rule used to allocate seats at the province level, and long is the shape of the results dataframe (as it does not have the political parties in the columns).

But to allocate seats per province, we need to know how many seats correspond to each province. We can do that for the November 2019 elections by calling:

In [56]:
seats_per_prov = sp.data.load_seats_by_province()

Now we can call the function that runs the D'Hondt rule. But let us first have a look at its signature:

In [57]:
help(sp.dhondt_rule_long)

Help on function dhondt_rule_long in module spanish_elections.dhondt_rule:

dhondt_rule_long(results: pandas.core.frame.DataFrame, n_seats: dict, province_col='province', party_col='party', votes_col='votes', seats_col='seats') -> pandas.core.frame.DataFrame
    Runs `dhondt_rule_long_single province` for each province in results dataframe.
    
    Never works in place.



We see that the function can be adapted to run with a dataframe with different column names. In our case, most default values are correct (e.g. votes_col is "votes"), except for province, so we will let the method know that the province column is actually named "provincia":

In [58]:
results_with_seats = sp.dhondt_rule_long(results, seats_per_prov, province_col='provincia')
results_with_seats.head()

,party,provincia,votes,total_seats,seats
0,PSOE,A Coruña,184178,8,3
1,PP,A Coruña,187127,8,3
2,VOX,A Coruña,50325,8,0
3,PODEMOS-IU,A Coruña,0,8,0
4,ERC-SOBIRANISTES,A Coruña,0,8,0


We appreciate a number of changes:
1. The order of results has been modified. This is due to an internal call to pandas groupby
2. Most importantly, two additional columns have been added: `total_seats` and `seats`. `total_seats` corresponds to the total number of seats per province and `seats` to the number of seats per political party in that province.

### Aggregate results

We can use the `sp.summarise.agg_results` to aggregate all the seats of the elections

In [59]:
agg_results = sp.summarise.agg_results(results_with_seats,
                                       result_col='seats')
agg_results

party
BNG                       1
CCa-PNC-NC                2
CUP-PR                    2
Cs                       10
EAJ-PNV                   6
ECP-GUANYEM EL CANVI      7
EH Bildu                  5
ERC-SOBIRANISTES         13
JxCAT-JUNTS               8
MÁS PAÍS-EQUO             2
MÉS COMPROMÍS             1
NA+                       2
PODEMOS-EU                2
PODEMOS-IU               26
PP                       89
PRC                       1
PSOE                    120
VOX                      52
¡TERUEL EXISTE!           1
Name: seats, dtype: int64

We can check that the results are correct by loading the seats from the actual elections:

In [60]:
actual_results = sp.data.load_results(type='seats')
actual_results = sp.summarise.agg_results(actual_results,
                                         result_col='seats')
actual_results

party
BNG                       1
CCa-PNC-NC                2
CUP-PR                    2
Cs                       10
EAJ-PNV                   6
ECP-GUANYEM EL CANVI      7
EH Bildu                  5
ERC-SOBIRANISTES         13
JxCAT-JUNTS               8
MÁS PAÍS-EQUO             2
MÉS COMPROMÍS             1
NA+                       2
PODEMOS-EU                2
PODEMOS-IU               26
PP                       89
PRC                       1
PSOE                    120
VOX                      52
¡TERUEL EXISTE!           1
Name: seats, dtype: int64

In [61]:
(actual_results == agg_results).all()

True

We got them! Finally, we should check that the sum of all seats corresponds to the total number of seats in Congreso (350):

In [62]:
agg_results.sum()

350

Nice, we haven't missed any seats! For more information about the results, you can check [here](https://es.wikipedia.org/wiki/XIV_legislatura_de_Espa%C3%B1a) (it is in Spanish, though)

In the next section, we will check how to use blocs to see how merging political parties would lead to different results in the elections :)

### Working with blocs

**Note!** The Bloc object is still under construction. Therefore, all code presented in this section is specially unstable!

Let us first load a list with all the political parties that earned seats:

In [63]:
pparties = sp.data.load_political_parties(with_rep=True)
pparties

['PSOE',
 'PP',
 'VOX',
 'PODEMOS-IU',
 'ERC-SOBIRANISTES',
 'Cs',
 'JxCAT-JUNTS',
 'ECP-GUANYEM EL CANVI',
 'EAJ-PNV',
 'EH Bildu',
 'MÁS PAÍS-EQUO',
 'CUP-PR',
 'PODEMOS-EU',
 'CCa-PNC-NC',
 'NA+',
 'MÉS COMPROMÍS',
 'BNG',
 'PRC',
 '¡TERUEL EXISTE!']

The `spanish_elections` package makes accessible a number of dictionaries that are helpful to work with Spanish political parties that earned at least a seat in the November 2019 elections. To access them, we can apply the `sp.data.load_blocs` function. Currently, this function loads a dictionary of dictionaries (a much more stable solution than loading a dictionary of Bloc objects)

In [64]:
all_blocs = sp.data.load_blocs()
all_blocs

{'investment_blocs_2020': {'PSOE': 'pro',
  'PP': 'against',
  'VOX': 'against',
  'PODEMOS-IU': 'pro',
  'ERC-SOBIRANISTES': 'abstain',
  'Cs': 'against',
  'JxCAT-JUNTS': 'against',
  'ECP-GUANYEM EL CANVI': 'pro',
  'EAJ-PNV': 'pro',
  'EH Bildu': 'abstain',
  'MÁS PAÍS-EQUO': 'pro',
  'CUP-PR': 'against',
  'PODEMOS-EU': 'pro',
  'CCa-PNC-NC': None,
  'NA+': 'against',
  'MÉS COMPROMÍS': 'pro',
  'BNG': 'pro',
  'PRC': 'against',
  '¡TERUEL EXISTE!': 'pro'}}

We see that there is a single dictionary present at the moment. It shows the political parties that voted pro, against or abstained to elect the current executive in Spain. The `spanish_elections` package creates a `Bloc` object, which is nothing but a wrapper for `dict` with some convenience functions such as checks for values or operations with DataFrames. Let us create a Bloc object with the dictionary above:

In [65]:
from spanish_elections.summarise import Bloc
investment_bloc = Bloc(all_blocs['investment_blocs_2020'])
investment_bloc

The original dictionary lies in the `classif` attribute:

In [66]:
investment_bloc.classif

{'PSOE': 'pro',
 'PP': 'against',
 'VOX': 'against',
 'PODEMOS-IU': 'pro',
 'ERC-SOBIRANISTES': 'abstain',
 'Cs': 'against',
 'JxCAT-JUNTS': 'against',
 'ECP-GUANYEM EL CANVI': 'pro',
 'EAJ-PNV': 'pro',
 'EH Bildu': 'abstain',
 'MÁS PAÍS-EQUO': 'pro',
 'CUP-PR': 'against',
 'PODEMOS-EU': 'pro',
 'CCa-PNC-NC': None,
 'NA+': 'against',
 'MÉS COMPROMÍS': 'pro',
 'BNG': 'pro',
 'PRC': 'against',
 '¡TERUEL EXISTE!': 'pro'}

Also, you can create your own bloc objects interactively and then save them to disk. The typical workflow to do that is presented here with an example in markdown:

```
investment_blocs_2020 = sp.summarise.create_bloc_interactively(pparties, 
                                                               bloc_values=['pro', 'against', 'abstain'])
# save bloc
investment_blocs_2020.save_bloc_as_json(filename, dir)
```

where the key function is `sp.summarise.create_bloc_interactively` that takes:
- the list of political parties you want to classify as first argument
- `bloc_values`, the values that the dictionary is restricted to have. 

Give it a try and create your own Blocs!

#### Convenience functions with Bloc

With Bloc, we can easily map the political party in a pandas dataframe to blocs:

In [67]:
investment_bloc.add_bloc_column(results, party_col='party', bloc_col='bloc')

,provincia,party,votes,bloc
0,Almería,PSOE,89295,pro
1,Cádiz,PSOE,188271,pro
2,Córdoba,PSOE,146761,pro
3,Granada,PSOE,160190,pro
4,Huelva,PSOE,91656,pro
...,...,...,...,...
6911,Alicante,UDT,0,NaN
6912,Castellón,UDT,0,NaN
6913,Valencia,UDT,0,NaN
6914,Ceuta,UDT,0,NaN


Where we have successfully added the bloc of each political party as a new column. Plus, we can aggregate by bloc as we wish with the following function:

In [68]:
results_with_blocs = investment_bloc.summarise_by_bloc(results, party_col='party',
                                                       other_agg=['provincia'],
                                                       agg_fun={'votes': sum})
results_with_blocs = results_with_blocs.reset_index()
results_with_blocs

,provincia,bloc,votes
0,A Coruña,abstain,0
1,A Coruña,against,265921
2,A Coruña,pro,332001
3,Albacete,abstain,0
4,Albacete,against,119088
...,...,...,...
151,Zaragoza,against,259263
152,Zaragoza,pro,216201
153,Ávila,abstain,0
154,Ávila,against,55959


#### Compute dhondt rule using a Bloc object

It would be interesting to know the counterfactual number of seats pro or against the current government if all parties in the investment blocs would have participated together in the election (assuming that voters would have not changed their votes if that had been so). In fact, using the previous dataframe and `sp.dhondt_rule_long` it is already trivial to do so:

In [69]:
results_by_blocs = sp.dhondt_rule_long(results_with_blocs, 
                                       seats_per_prov, 
                                       province_col='provincia',
                                      party_col='bloc')
results_by_blocs

,bloc,provincia,votes,total_seats,seats
0,abstain,A Coruña,0,8,0
1,against,A Coruña,265921,8,4
2,pro,A Coruña,332001,8,4
3,abstain,Albacete,0,4,0
4,against,Albacete,119088,4,2
...,...,...,...,...,...
151,against,Zaragoza,259263,7,4
152,pro,Zaragoza,216201,7,3
153,abstain,Ávila,0,3,0
154,against,Ávila,55959,3,2


We find the results disaggregated by province. Let us use the `sp.summarise.agg_results` function to obtain the total result:

In [70]:
sp.summarise.agg_results(results_by_blocs, result_col='seats',
                        party_col='bloc')

bloc
abstain     15
against    176
pro        159
Name: seats, dtype: int64

We find something quite striking: there would have been an alternative majority had not been for the partisan composition of the Congreso de Diputados. This demonstrates the importance of taking into account Dhond't method applied at the province level when analysing counterfactual scenarios for the Spanish elections.

### Transferring votes

Transferring votes is more flexible than using blocks. The downside is that you need to design your own transferences.

First, let me import and show the help for the main function to transfer votes, `transfer_votes_long`

In [71]:
from spanish_elections import transfer_votes_long

In [72]:
help(transfer_votes_long)

Help on function transfer_votes_long in module spanish_elections.transfer_votes:

transfer_votes_long(results: pandas.core.frame.DataFrame, list_transfer_votes, province_col='province', party_col='party', votes_col='votes', replace=False, final_votes_col='final_votes')



The main parameters of `transfer_votes_long` are `results` and `list_transfer_votes`. `results` is a long dataframe with province, party and the number of votes for each party in that province (i.e. the `results` dataframe in this notebook). `list_transfer_votes` is a list of tuples of length 3, with the following structure:

`(source_party, destination_party, percentage of votes from source_party to destination_party to be transferred)`

#### Using transfers to simulate a reallocation of votes in the right

We can simulate the counterfactual scenario where Ciudadanos (Cs) and VOX disappeared, and all their votes went to the oldest right-wing political party, the Popular Party. What would be the consequences for the composition of the current Congreso de Diputados? Let us explore it:

In [73]:
all_right_to_pp_transfer_list = [('Cs', 'PP', 1.), ('VOX', 'PP', 1.)]

Where we see that all votes from Cs and VOX would be transferred to PP. Let us now apply this transfer list to the results of the elections:

In [74]:
all_right_to_pp_results = \
transfer_votes_long(results, all_right_to_pp_transfer_list, province_col='provincia')
all_right_to_pp_results.head()

,party,provincia,votes,final_votes
0,PSOE,A Coruña,184178,184178
1,PP,A Coruña,187127,265921
2,VOX,A Coruña,50325,0
3,PODEMOS-IU,A Coruña,0,0
4,ERC-SOBIRANISTES,A Coruña,0,0


And we will now check how the results would look for my province, Córdoba:

In [75]:
all_right_to_pp_results.query('provincia == "Córdoba" & final_votes > 0')

,party,provincia,votes,final_votes
1139,PSOE,Córdoba,146761,146761
1140,PP,Córdoba,99999,218762
1142,PODEMOS-IU,Córdoba,64769,64769
1158,PACMA,Córdoba,3946,3946
1160,RECORTES CERO-GV,Córdoba,549,549
1161,PUM+J,Córdoba,478,478
1164,AxSÍ,Córdoba,1019,1019
1165,PCPE,Córdoba,544,544
1166,PCTE,Córdoba,321,321
1169,PCOE,Córdoba,1062,1062


We observe that all parties with votes have the same votes except for the PP, that now would achieve many more votes. Just as we expected! ;)

Getting the final result of the elections is as simple as running `dhont_rule_long` specifying the new column with votes.

In [76]:
all_right_to_pp_seats = \
sp.dhondt_rule_long(all_right_to_pp_results, seats_per_prov, province_col='provincia',
                   votes_col='final_votes')
all_right_to_pp_seats.head()

,party,provincia,votes,final_votes,total_seats,seats
0,PSOE,A Coruña,184178,184178,8,3
1,PP,A Coruña,187127,265921,8,4
2,VOX,A Coruña,50325,0,8,0
3,PODEMOS-IU,A Coruña,0,0,8,0
4,ERC-SOBIRANISTES,A Coruña,0,0,8,0


Where we note that the column seats got the seats for each political party in each province. Finally, we summarise the results:

In [77]:
all_right_to_pp_aggregate_results = sp.summarise.agg_results(all_right_to_pp_seats,
                                                             result_col='seats')
all_right_to_pp_aggregate_results

party
CCa-PNC-NC                2
CUP-PR                    2
EAJ-PNV                   6
ECP-GUANYEM EL CANVI      6
EH Bildu                  4
ERC-SOBIRANISTES         13
JxCAT-JUNTS               8
MÁS PAÍS-EQUO             2
MÉS COMPROMÍS             1
NA+                       2
PODEMOS-EU                2
PODEMOS-IU               24
PP                      175
PRC                       1
PSOE                    101
¡TERUEL EXISTE!           1
Name: seats, dtype: int64

Now, taking into account that NA+ is a coalition of PP and Cs in Navarra, the bloc of the right would have won the elections (with 177 seats) if they had participated together in the elections. Of course, this result only holds under the assumption that no votes of the right would have leaked if that was case.

The last assumption is utterly unrealistic. The nice part about the `transfer_votes_long` function, however, is that you can specify any proportion of votes in the transfer list, so you can make your transfers as realistic as you want. In fact, in the future I expect to incorporate tools to analyse microdata from Centro de Investigaciones Sociológicas (CIS). Then, we will be able to write much more realistic transferences.